# Modello LSMT (Long Short-Term Memory) 

Primo step split del dataset

In [ ]:
import glob
import os



In [ ]:
INPUT_DIR = '../Pre-Elaborazione Dati/Dataset'
all_stitched_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

# 2. Definisci la suddivisione 16-4-4 (8-2-2 mesi)
# (Assicurati che i tuoi file siano ordinati cronologicamente, 
#  'blocco_000', 'blocco_001', ecc. come abbiamo fatto)

TRAIN_FILES = all_stitched_files[0:16]  # Primi 16 blocchi (8 mesi)
VAL_FILES = all_stitched_files[16:20] # Successivi 4 blocchi (2 mesi)
TEST_FILES = all_stitched_files[20:24] # Ultimi 4 blocchi (2 mesi)

print(f"File di Training: {len(TRAIN_FILES)}")
print(f"File di Validazione: {len(VAL_FILES)}")
print(f"File di Test: {len(TEST_FILES)}")

# Stampa un esempio
print(f"\nPrimo file di Train: {os.path.basename(TRAIN_FILES[0])}")
print(f"Primo file di Val: {os.path.basename(VAL_FILES[0])}")
print(f"Primo file di Test: {os.path.basename(TEST_FILES[0])}")